## Preparing workspace

Libraries installation

In [1]:
#!pip install --upgrade pip setuptools wheel
#!pip install xarray earthengine-api geemap numpy pandas geopandas 

In [2]:
# Google Earth Engine Libraries
import ee
import geemap

# Basic Libraries
import json
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd


Authentication in Google Earth Engine, pointing to the project id in Google Cloud

In [3]:
# Initialize the Earth Engine API
ee.Authenticate()   # Opens a link to get a code for authentication, add the code in the dialog box in the command palette
ee.Initialize(project='awesome-habitat-472614-m0')

## Exemplo: usando imagens LANDSAT

In [4]:
Map = geemap.Map()

In [5]:
# Exemplo: Selecionar uma coleção de imagens do Landsat
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')

# Definir uma região de interesse
aoi = ee.Geometry.Point([-159.5, 20.8]).buffer(10000) # Exemplo: Ilha Kauai

# Filtrar por região e percentual de nuvem
filtered = landsat.filterBounds(aoi).filterMetadata('CLOUD_COVER', 'less_than', 10)

# Selecionar a primeira imagem da coleção
image = filtered.first()

# Adicionar a imagem ao mapa
Map.addLayer(image, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 3000}, 'Landsat 8 Image')

# Centralizar o mapa na área de interesse
Map.centerObject(aoi, 10)

In [6]:
Map

Map(center=[20.800010258872835, -159.4999996956733], controls=(WidgetControl(options=['position', 'transparent…

## Exemplo: Rio de Grande Sul

In [7]:
rs_geojson_path = Path("Rio_Grande_do_Sul_Brazil.geojson")

with rs_geojson_path.open("r", encoding="utf-8") as brazil_file:
    rs_geojson = json.load(brazil_file)

# Remove the 'id' property from features as Earth Engine expects 'system:index' to be a string.
for feature in rs_geojson.get('features', []):
    feature['id'] = str(feature.get('id')) # Convert id to string
    # Alternatively, you could remove the 'id' property completely if it's not needed.
    # if 'id' in feature:
    #    del feature['id']

Map = geemap.Map()

region_name = "Rio Grande do Sul"
roi = geemap.geojson_to_ee(geo_json=rs_geojson, geodesic=True)

# Define pre-flood and post-flood datesd dates
pre_flood_start_date = "2024-01-01"
pre_flood_end_date = "2024-04-27"
post_flood_start_date = "2024-04-29"
post_flood_end_date = "2024-05-17"

In [8]:
# Filter Sentinel-1 data for the pre-flood period
pre_flood_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(pre_flood_start_date, pre_flood_end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .select('VV')

# Filter Sentinel-1 data for the post-flood period
post_flood_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterDate(post_flood_start_date, post_flood_end_date) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.eq('resolution_meters', 10)) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .select('VV')

# Reduce collections to a single image (e.g., median composite)
pre_flood_image = pre_flood_collection.median().clip(roi)
post_flood_image = post_flood_collection.median().clip(roi)

In [9]:
# Calculate the difference between pre- and post-flood images
difference = pre_flood_image.subtract(post_flood_image)

# Define a threshold to identify flooded areas
# This value may need to be adjusted based on the specific data
flood_threshold = 1.5

# Create a flood extent mask
flood_extent = difference.gt(flood_threshold)

# Mask out areas that are permanent water bodies or were already flooded
# For a more robust analysis, you can use a water mask dataset
# like JRC Global Surface Water.
permanent_water = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('occurrence').gt(90)
flood_extent = flood_extent.And(permanent_water.eq(0))

In [10]:
# Add layers to the map for visualization
m = geemap.Map()
m.add_basemap('SATELLITE')
m.centerObject(roi, 8)
m.addLayer(pre_flood_image, {'min': -20, 'max': -5}, 'Pre-Flood Image')
m.addLayer(post_flood_image, {'min': -20, 'max': -5}, 'Post-Flood Image')
m.addLayer(flood_extent.updateMask(flood_extent), {'palette': 'blue'}, 'Flood Extent')
m.addLayer(roi, {}, 'Rio Grande do Sul ROI', False)
m

Map(center=[-29.778660140568665, -53.24534198340686], controls=(WidgetControl(options=['position', 'transparen…

# Disaster Detector V2

Feito com ajuda do Codex, ChatGPT e NotebookLM.

Links:

- https://notebooklm.google.com/notebook/6f5a37f5-df11-4765-9b3b-582400e0270a
- https://chatgpt.com/share/68e11d4e-1e90-8012-8902-fd6844200085
- Não consigo compartilhar o Codex (ou não sei como)

## 1. Study Region
- Load `geojson_area_brazil.json`.
- Convert to ee.FeatureCollection and display.

In [11]:
geojson_path = Path("geojson_area_brazil.json")

with geojson_path.open("r", encoding="utf-8") as brazil_file:
    brazil_geojson = json.load(brazil_file)

# Remove the 'id' property from features as Earth Engine expects 'system:index' to be a string.
for feature in brazil_geojson.get('features', []):
    feature['id'] = str(feature.get('id')) # Convert id to string
    # Alternatively, you could remove the 'id' property completely if it's not needed.
    # if 'id' in feature:
    #    del feature['id']


region_fc = geemap.geojson_to_ee(brazil_geojson)
region_geometry = region_fc.geometry()

Map = geemap.Map()
Map.center_object(region_geometry, 4)
Map.addLayer(region_geometry, {"color": "FF0000"}, "Brazil AOI")
Map

Map(center=[-10.632060280685165, -53.21012571368745], controls=(WidgetControl(options=['position', 'transparen…

## 2. Helper Functions
- Create functions for temporal filtering, mosaicking, resampling.
- Reusable reducers for monthly/annual composites.


In [12]:
# 2. Helper Functions
def make_date_range(start_date, end_date, freq):
    """Return ISO date strings sampled at the requested frequency."""
    dates = pd.date_range(start=start_date, end=end_date, freq=freq, inclusive="both")
    return [str(date.date()) for date in dates]

def build_time_bands(collection_id, start_date, end_date, freq, reducer, scale):
    """Generate temporal composites (monthly or annual) clipped to the region."""
    ee_collection = ee.ImageCollection(collection_id).filterDate(start_date, end_date).map(
        lambda img: img.clip(region_geometry)
    )
    date_list = make_date_range(start_date, end_date, freq)
    composite_stack = []
    for iso_date in date_list:
        period_start = iso_date
        period_end = pd.to_datetime(iso_date) + pd.tseries.frequencies.to_offset(freq) - pd.Timedelta(days=1)
        ee_period = ee_collection.filterDate(period_start, str(period_end.date()))
        composite_image = ee_period.reduce(reducer).reproject(crs="EPSG:4326", scale=scale)
        composite_stack.append(composite_image.set("system:time_start", ee.Date(period_start).millis()))
    return ee.ImageCollection(ee.List(composite_stack))


## 3. Terrain & Hydrology
- Fetch NASADEM and HydroSHEDS.
- Derive slope, aspect, roughness, HAND, TWI, distance to drainage.

In [13]:
# 3. Terrain & Hydrology
nasadem_img = ee.Image("NASA/NASADEM_HGT/001").select("elevation").clip(region_geometry)

terrain_products = ee.Terrain.products(nasadem_img)
slope_deg = terrain_products.select("slope")   # degrees
aspect_deg = terrain_products.select("aspect")

# Convert slope to radians for TWI calculation
slope_rad = slope_deg.multiply(3.141592653589793/180.0)

# HydroSHEDS (IDs correct)
flow_acc = ee.Image("WWF/HydroSHEDS/15ACC").clip(region_geometry)  # upstream pixel count 15"
flow_dir = ee.Image("WWF/HydroSHEDS/15DIR").clip(region_geometry)  # directions 15"

# Distance to main channel (using distance method on drainage mask)
# 1) Define channels by accumulation threshold
drainage_mask = flow_acc.gt(1000)  # adjust threshold
# 2) Calculate distance from non-channel pixels to channel pixels
distance_to_channel = drainage_mask.Not().fastDistanceTransform(30).sqrt().multiply(30)
distance_to_channel = distance_to_channel.rename('dist_to_channel_m')


# HAND (MERIT Hydro)
merit = ee.Image('MERIT/Hydro/v1_0_1')
hand  = merit.select('hnd').rename('hand_m').clip(region_geometry)          # HAND (m)
upa   = merit.select('upa').rename('upa_km2').clip(region_geometry)         # upstream area (km²)

# TWI = ln( a / tan(beta) )
# a = specific contributing area (m² per unit contour length). Using upa_km2 as proxy.
twi = (upa.multiply(1e6).add(1)).log().subtract(slope_rad.tan().add(1e-6).log()) \
        .rename('twi_proxy')

## 4. Impervious Surface & Exposure
- GMIS annual impervious fraction.
- Population density & built infrastructure proxies.


In [14]:
# --- Impervious Surface & Exposure (2005–2025) ---

# Fontes
GAIA_ID   = 'Tsinghua/FROM-GLC/GAIA/v10'  # 1985–2018 (ano de conversão)  [30 m]
GISD30_ID = 'projects/sat-io/open-datasets/GISD30_1985_2020'              # 1985–2020 (labels por períodos) [30 m]
ESRI_TS   = 'projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS'  # 2017–2024 (classe 5=Built) [10 m]
# GHSL por ano disponível (100 m; 5/5 anos; tem 2025)
def ghsl_built_fraction_for_year(y):
    y = int(y)
    ghsl_path = f'JRC/GHSL/P2023A/GHS_BUILT_S/{y}'
    ghsl = ee.Image(ghsl_path).select('built_surface').clip(region_geometry)
    # 100m pixel tem 100x100 = 10.000 m² como referência do grid GHSL
    return ghsl.divide(10000.0).rename('impervious_fraction')

gaia_img = ee.Image(GAIA_ID).clip(region_geometry)
gisd30   = ee.Image(GISD30_ID).clip(region_geometry)
esri_ic  = ee.ImageCollection(ESRI_TS).filterBounds(region_geometry)

def gaia_mask_for_year(year):
    """Pixel é impervious se ano de conversão <= year (0/1)."""
    conv_year = ee.Image(2019).subtract(gaia_img).rename('conv_year')  # 2019 - GAIA
    return conv_year.lte(year).rename('imp_bin').toFloat()

def gisd30_mask_for_year(year):
    """
    GISD30 rotula: 0=pervious, 1=antes85, 2=85–90,3=90–95,4=95–00,5=00–05,6=05–10,7=10–15,8=15–20.
    Para um ano Y, incluímos todos os códigos <= classe do período correspondente.
    """
    y = int(year)
    # mapeia ano -> código máximo
    code = (
        1 if y <= 1984 else
        2 if y <= 1989 else
        3 if y <= 1994 else
        4 if y <= 1999 else
        5 if y <= 2004 else
        6 if y <= 2009 else
        7 if y <= 2014 else
        8  # 2015–2020
    )
    return gisd30.gt(0).And(gisd30.lte(code)).rename('imp_bin').toFloat()

def esri_built_mask_for_year(year):
    """Retorna 0/1 para Built Area (classe 5) no ESRI LULC do ano. Faz fallback por índice."""
    y = int(year)
    ic = esri_ic.filter(ee.Filter.eq('year', y))
    img = ee.Image(ee.Algorithms.If(ic.size().gt(0), ic.first(),
             esri_ic.filter(ee.Filter.stringContains('system:index', ee.Number(y).format())).first()))
    # Classe 5 = Built Area (versão remapeada do catálogo)
    built = ee.Image(img).select(0).eq(5).rename('imp_bin').toFloat()
    return built

def impervious_fraction_for_year(year):
    """Combina fontes disponíveis e retorna fração 0..1 para o ano."""
    y = int(year)
    out = ee.Image(0).rename('impervious_fraction').toFloat()

    # GAIA até 2018
    out = ee.Image(ee.Algorithms.If(y <= 2018,
            out.max(gaia_mask_for_year(y)), out))

    # ESRI 2017–2024 (10 m)
    out = ee.Image(ee.Algorithms.If((y >= 2017) & (y <= 2024),
            out.max(esri_built_mask_for_year(y)), out))

    # GISD30 reforço para 2019–2020
    out = ee.Image(ee.Algorithms.If((y >= 2019) & (y <= 2020),
            out.max(gisd30_mask_for_year(y)), out))

    # GHSL para anos quinquenais; usamos 2025 explicitamente
    out = ee.Image(ee.Algorithms.If(y == 2025,
            out.max(ghsl_built_fraction_for_year(y)), out))

    return out.set('year', y).clip(region_geometry)

# Construir a coleção anual 2005–2025
years = list(range(2005, 2026))
imperv_list = [impervious_fraction_for_year(y) for y in years]
impervious_series_ic = ee.ImageCollection(imperv_list)


In [15]:
# -----------------------------
# Exposure (População 2005–2025 + Land Cover)
# -----------------------------

# 1) GPW v4.11 Population Count (anos-âncora: 2005, 2010, 2015, 2020)
GPW_COLL_ID = 'CIESIN/GPWv411/GPW_Population_Count'
gpw_ic = ee.ImageCollection(GPW_COLL_ID).filterBounds(region_geometry)

def gpw_year_image(year):
    """Retorna a imagem GPW do ano pedido (se existir) já recortada."""
    return (gpw_ic
            .filter(ee.Filter.calendarRange(year, year, 'year'))
            .first()
            .rename('pop')
            .toFloat()
            .clip(region_geometry))

ANCHOR_YEARS = [2005, 2010, 2015, 2020]
anchor_imgs = {y: gpw_year_image(y) for y in ANCHOR_YEARS}

def interp_log_linear(img0, img1, y0, y1, y):
    """
    Interpolação multiplicativa (log-linear) entre y0 e y1:
    pop(y) = pop0 * (pop1/pop0)^t, onde t = (y - y0)/(y1 - y0).
    Evita valores negativos e segue melhor a dinâmica populacional.
    """
    t = (y - y0) / float(y1 - y0)
    ratio = img1.divide(img0.max(1e-6))  # protege contra zero
    growth = ratio.pow(t)
    return img0.multiply(growth).rename('pop')

def project_forward(img_prev, img_next, y_prev, y_next, y):
    """
    Projeta para frente usando CAGR no intervalo [y_prev, y_next]
    e avança a partir de img_next até o ano y.
    """
    years_span = float(y_next - y_prev)   # tipicamente 5
    cagr = img_next.divide(img_prev.max(1e-6)).pow(1.0 / years_span)  # fator anual
    years_ahead = float(y - y_next)
    return img_next.multiply(cagr.pow(years_ahead)).rename('pop')

def population_image_for_year(y):
    y = int(y)
    if y in ANCHOR_YEARS:
        return anchor_imgs[y].set('year', y)

    if 2005 < y < 2010:
        return interp_log_linear(anchor_imgs[2005], anchor_imgs[2010], 2005, 2010, y).set('year', y)
    if 2010 < y < 2015:
        return interp_log_linear(anchor_imgs[2010], anchor_imgs[2015], 2010, 2015, y).set('year', y)
    if 2015 < y < 2020:
        return interp_log_linear(anchor_imgs[2015], anchor_imgs[2020], 2015, 2020, y).set('year', y)
    if 2020 < y <= 2025:
        # projeta 2021–2025 com CAGR 2015→2020
        return project_forward(anchor_imgs[2015], anchor_imgs[2020], 2015, 2020, y).set('year', y)

    # fallback (não deve ocorrer no seu intervalo)
    return anchor_imgs[2005].set('year', y)

years_pop = list(range(2005, 2026))
population_series_ic = ee.ImageCollection([population_image_for_year(y) for y in years_pop])



In [16]:
# -----------------------------
# Land Cover anual (2005–2025)
# -----------------------------

# Fontes
MCD12Q1_ID = 'MODIS/061/MCD12Q1'   # 500 m, 2001–2020 (LC_Type1 IGBP)
MCD12C1_ID = 'MODIS/061/MCD12C1'   # 0.05°, 2001–2023 (LC_Type1 IGBP)
ESRI_TS_ID = 'projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS'  # 2017–2024

mcd12q1 = ee.ImageCollection(MCD12Q1_ID).filterBounds(region_geometry)
mcd12c1 = ee.ImageCollection(MCD12C1_ID).filterBounds(region_geometry)
esri_ts = ee.ImageCollection(ESRI_TS_ID).filterBounds(region_geometry)

def mcd12q1_year(y):
    """Retorna LC_Type1 (IGBP) 500 m para o ano y, se existir."""
    return (mcd12q1
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .first()
            .select('LC_Type1')
            .clip(region_geometry))

def mcd12c1_year(y):
    """Retorna LC_Type1 (IGBP) 0.05° para o ano y, se existir."""
    return (mcd12c1
            .filter(ee.Filter.calendarRange(y, y, 'year'))
            .first()
            .select('LC_Type1')
            .clip(region_geometry))

def esri_year(y):
    """Retorna classe Esri 10 m do ano y (primeiro band)."""
    ic = esri_ts.filter(ee.Filter.eq('year', y))
    img = ee.Image(ee.Algorithms.If(ic.size().gt(0), ic.first(),
             esri_ts.filter(ee.Filter.stringContains('system:index', ee.Number(y).format())).first()))
    return ee.Image(img).select(0).clip(region_geometry)

def esri_to_igbp(esri_img):
    """
    Mapeamento aproximado Esri -> IGBP (LC_Type1):
      1 Water -> 0 Water
      2 Trees -> 5 Mixed Forest (proxy)
      3 Grass -> 10 Grasslands
      4 Flooded vegetation -> 11 Permanent Wetlands
      5 Built Area -> 13 Urban and Built-up
      6 Crops -> 12 Croplands
      7 Bare ground -> 16 Barren
      8 Snow/Ice -> 15 Snow and Ice
      9 Clouds -> 254 Unclassified
      10 Shrubland -> 6 Closed Shrublands (proxy)
      11 Rangeland -> 10 Grasslands (proxy)
    """
    src = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
    dst = [0, 5,10,11,13,12,16,15,254, 6, 10]
    return esri_img.remap(src, dst).rename('LC_Type1')

def land_cover_for_year(y):
    """Prioriza MODIS (IGBP) onde existe; usa Esri reclassificado para 2024; 2025 = LOCF."""
    y = int(y)
    lc = ee.Image(None)

    # 2005–2020: MCD12Q1
    lc = ee.Image(ee.Algorithms.If((y >= 2005) & (y <= 2020),
        mcd12q1_year(y), lc))

    # 2021–2023: MCD12C1 (IGBP coarse) como extensão
    lc = ee.Image(ee.Algorithms.If((y >= 2021) & (y <= 2023),
        mcd12c1_year(y), lc))

    # 2024: Esri 10 m reclassificado para IGBP
    lc = ee.Image(ee.Algorithms.If(y == 2024,
        esri_to_igbp(esri_year(2024)), lc))

    # 2025: projeta como cópia de 2024 (LOCF)
    lc = ee.Image(ee.Algorithms.If(y == 2025,
        esri_to_igbp(esri_year(2024)), lc))

    return ee.Image(lc).rename('LC_Type1').set('year', y)

years_lc = list(range(2005, 2026))
lc_images = [land_cover_for_year(y) for y in years_lc]
land_cover_series_ic = ee.ImageCollection(lc_images)



## 5. Radar Backscatter Texture
- Sentinel-1 VV/VH monthly medians.
- VV/VH ratio, GLCM entropy/energy.


In [17]:
# 5. Radar Backscatter Texture (2005–2025) com fallback PALSAR

# Sentinel-1 (C-band) — disponível 2014/2015+ no GEE
s1_ic = (ee.ImageCollection("COPERNICUS/S1_GRD")
         .filterBounds(region_geometry)
         .filter(ee.Filter.eq("instrumentMode", "IW"))
         .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
         .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")))

# PALSAR/PALSAR-2 mosaics anuais (L-band) — 2007–2010 e 2015–2023
palsar_v1   = ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR")
palsar_epoch= ee.ImageCollection("JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH")
palsar_ic   = palsar_v1.merge(palsar_epoch).filterBounds(region_geometry)  # 2007–2010, 2015–2023

# anos disponíveis em PALSAR (server-side)
palsar_years = ee.List(palsar_ic.aggregate_array('year').distinct()).sort()

def palsar_nearest_year(y):
    """Retorna o ano PALSAR mais próximo (server-side)."""
    y = ee.Number(y)
    diffs = palsar_years.map(lambda a: ee.Number(a).subtract(y).abs())
    minv  = ee.Number(diffs.reduce(ee.Reducer.min()))
    idx   = diffs.indexOf(minv)
    return ee.Number(palsar_years.get(idx))

def palsar_year_img(y):
    """Imagem PALSAR do ano (ou a mais próxima, se não houver)."""
    y_nn = palsar_nearest_year(y)
    cand = palsar_ic.filter(ee.Filter.eq('year', y_nn))
    return ee.Image(cand.first()).clip(region_geometry)

def palsar_metrics_for_year(y):
    """
    Converte HH/HV DN -> dB (~γ0) com 20*log10(DN) - 83 (JAXA),
    calcula razão e textura GLCM (size=3) sobre HH.
    """
    img = palsar_year_img(y)
    # converter para dB
    hh = img.select('HH')
    hv = img.select('HV')
    hh_db = hh.log10().multiply(20).subtract(83.0)
    hv_db = hv.log10().multiply(20).subtract(83.0)
    ratio = hh_db.subtract(hv_db).rename('sar_ratio_db')
    # textura sobre HH em 8-bit (faixa típica -25..5 dB)
    hh8 = hh_db.unitScale(-25, 5).multiply(255).toUint8()
    glcm = hh8.glcmTexture(size=3)
    entropy = glcm.select('HH_entropy').rename('sar_entropy')
    energy  = glcm.select('HH_energy' ).rename('sar_energy')
    return ee.Image.cat([ratio, entropy, energy]) \
            .set('sar_source', 'PALSAR')

def s1_metrics_for_month(start, end):
    """
    Mediana mensal S1 (VV/VH em dB), razão VV–VH e textura GLCM sobre VV.
    """
    monthly = s1_ic.filterDate(start, end)
    median  = monthly.median().clip(region_geometry)
    vv = median.select('VV')
    vh = median.select('VH')
    ratio = vv.subtract(vh).rename('sar_ratio_db')  # dB-dB ~ log-ratio
    vv8 = vv.unitScale(-25, 5).multiply(255).toUint8()
    glcm = vv8.glcmTexture(size=3)
    entropy = glcm.select('VV_entropy').rename('sar_entropy')
    energy  = glcm.select('VV_energy' ).rename('sar_energy')
    return ee.Image.cat([ratio, entropy, energy]) \
            .set('sar_source', 'S1')

# constrói uma coleção mensal 2005–2025, com fallback PALSAR quando não houver S1
def make_radar_month(iso):
    start = ee.Date(iso)
    end   = start.advance(1, "month")
    # há S1 no mês?
    has_s1 = s1_ic.filterDate(start, end).size().gt(0)
    # se houver S1, usa S1; senão usa PALSAR do ano mais próximo
    y = start.get('year')
    im = ee.Image(ee.Algorithms.If(
        has_s1,
        s1_metrics_for_month(start, end),
        palsar_metrics_for_year(y)
    ))
    return im.set("system:time_start", start.millis())

# Lista mensal 2005–2025
radar_monthly = ee.ImageCollection(
    ee.List(make_date_range("2005-01-01", "2025-12-01", "MS"))
      .map(lambda d: make_radar_month(d))
)


## 6. Rainfall, Soil Moisture, TWS
- GPM IMERG monthly sums and rainfall probabilities.
- SMAP soil moisture and GRACE terrestrial water storage anomalies.


In [18]:
# 6. Rainfall (como está), Soil Moisture (blend FLDAS+SMAP) e TWS (GRACE/GRACE-FO)

# -------------------------
# 6A) Rainfall (já OK)
gpm_monthly = build_time_bands(
    collection_id="NASA/GPM_L3/IMERG_MONTHLY_V06",
    start_date="2005-01-01",
    end_date="2025-12-31",
    freq="MS",
    reducer=ee.Reducer.sum(),
    scale=10000,
)

def calc_rain_probability(collection, threshold):
    """Share de dias acima do limiar de chuva (ex.: 20 mm)."""
    daily = ee.ImageCollection(collection).filterDate("2005-01-01", "2025-12-31").map(
        lambda img: img.select("precipitationCal").gt(threshold).clip(region_geometry)
    )
    return daily.reduce(ee.Reducer.mean()).rename(f"rain_prob_gt_{threshold}mm")

rain_prob_20mm = calc_rain_probability("NASA/GPM_L3/IMERG_V06", 20)

# -------------------------
# 6B) Soil Moisture 2005–2025: FLDAS (2005–2015-03) + SMAP L4 (2015-04–2025), com correção de viés

# Helper: agrega coleção para médias mensais no AOI (sem reproject agressivo)
def monthly_mean(ic, band, start_date, end_date):
    months = ee.List(make_date_range(start_date, end_date, "MS"))
    def _one_month(iso):
        s = ee.Date(iso)
        e = s.advance(1, "month")
        img = ic.filterDate(s, e).select(band).mean().clip(region_geometry)
        return img.rename(band).set("system:time_start", s.millis())
    return ee.ImageCollection(months.map(_one_month))

# FLDAS monthly (2005–2015-03), banda em fração volumétrica
fldas_ic = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001").map(lambda im: im.clip(region_geometry))
fldas_sm_monthly = monthly_mean(
    fldas_ic, "SoilMoi00_10cm_tavg",
    "2005-01-01", "2015-03-01"
)

# SMAP L4 (3h) -> monthly mean (2015-04–2025-12), banda em fração volumétrica
smap_ic = ee.ImageCollection("NASA/SMAP/SPL4SMGP/008").map(lambda im: im.clip(region_geometry))
# agrega por mês
def _tag_month(im):
    d = ee.Date(im.get("system:time_start"))
    return im.set("month_start", d.update(day=1).format("YYYY-MM-01"))

smap_tagged = smap_ic.select("sm_surface").map(_tag_month)

def _agg_month(mstr):
    s = ee.Date(mstr)
    e = s.advance(1, "month")
    return (smap_tagged
            .filterDate(s, e)
            .mean()
            .rename("sm_surface")
            .set("system:time_start", s.millis()))

smap_months = ee.List(make_date_range("2015-04-01", "2025-12-01", "MS"))
smap_sm_monthly = ee.ImageCollection(smap_months.map(_agg_month))

# Correção de viés FLDAS -> SMAP usando a média do overlap 2015-04..2016-12 (por pixel)
fldas_overlap = fldas_sm_monthly.filterDate("2015-04-01", "2016-12-31").mean()
smap_overlap  = smap_sm_monthly.filterDate("2015-04-01", "2016-12-31").mean()
bias_img = smap_overlap.select("sm_surface").subtract(fldas_overlap.select("SoilMoi00_10cm_tavg"))

fldas_sm_adj = fldas_sm_monthly.map(
    lambda im: im.select("SoilMoi00_10cm_tavg").add(bias_img).rename("soil_moisture")
)
smap_sm_ren = smap_sm_monthly.map(lambda im: im.select("sm_surface").rename("soil_moisture"))

# Coleção final 2005–2025, banda 'soil_moisture' (fração volumétrica 0–1)
soil_moisture_monthly = fldas_sm_adj.merge(smap_sm_ren).sort("system:time_start")

# -------------------------
# 6C) Total Water Storage (GRACE/GRACE-FO) 2005–2025 com preenchimento de lacunas

grace_ic_raw = (ee.ImageCollection("NASA/GRACE/MASS_GRIDS_V04/MASCON")
                .select("lwe_thickness")
                .map(lambda im: im.clip(region_geometry)))

def grace_month_or_fill(iso):
    """Retorna a imagem do mês; se não houver, média do mês anterior e posterior."""
    s = ee.Date(iso)
    e = s.advance(1, "month")
    this = grace_ic_raw.filterDate(s, e)
    def _filled():
        prev = grace_ic_raw.filterDate("2002-01-01", s) \
                           .sort("system:time_start", False).first()
        nxt  = grace_ic_raw.filterDate(e, "2030-01-01") \
                           .sort("system:time_start").first()
        # média simples prev/next como fallback
        return ee.ImageCollection([prev, nxt]).mean().set("system:time_start", s.millis())
    out = ee.Image(ee.Algorithms.If(this.size().gt(0),
                                     ee.Image(this.first()).set("system:time_start", s.millis()),
                                     _filled()))
    return out.rename("tws_cm")  # mantém unidade do catálogo (cm)

grace_months = ee.List(make_date_range("2005-01-01", "2025-12-01", "MS"))
grace_tws_monthly = ee.ImageCollection(grace_months.map(grace_month_or_fill))


## 7. Surface Water & Flood Validation
- JRC Monthly surface water, recurrence.
- Global Flood Database events; WRI Aqueduct hazards for validation.


In [19]:
# 7. Surface Water & Flood Validation — 2005–2025 (mensal quando possível)

# -----------------------------
# A) JRC Global Surface Water (1984–2021)
# -----------------------------
gsw_monthly_ic = (
    ee.ImageCollection("JRC/GSW1_4/MonthlyHistory")
      .filterBounds(region_geometry)
)

gsw_layers = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").clip(region_geometry)
water_occurrence = gsw_layers.select("occurrence")   # [%]
water_recurrence = gsw_layers.select("recurrence")   # [%]
seasonality = gsw_layers.select("seasonality")       # [0..12]

def _water_bitmask_to_binary(img):
    water_flag = img.select("water")
    water_mask = water_flag.bitwiseAnd(3).eq(2).rename("water_mask")
    return (water_mask
            .copyProperties(img, ['system:time_start','year','month'])
            .set('year', img.get('year'))
            .set('month', img.get('month')))

def jrc_monthly_water_mask(start_date, end_date):
    return (gsw_monthly_ic
            .filterDate(start_date, end_date)
            .map(_water_bitmask_to_binary)
            .map(lambda im: im.clip(region_geometry)))

jrc_water_masks_2005_2021 = jrc_monthly_water_mask("2005-01-01", "2021-12-31")


# -----------------------------
# B) Extensão 2022–2025: Sentinel-1 + Landsat (opcional LANCE)
# -----------------------------

# --- Sentinel-1 mensal (2015+) -> água por backscatter baixo ---
s1_ic = (ee.ImageCollection("COPERNICUS/S1_GRD")
         .filterBounds(region_geometry)
         .filter(ee.Filter.eq("instrumentMode", "IW"))
         .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
         .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VH")))

def s1_water_for_month(iso):
    s = ee.Date(iso); e = s.advance(1, "month")
    col = s1_ic.filterDate(s, e)
    # Se não houver S1 no mês, devolve imagem vazia (será completada por Landsat)
    def _empty():
        return ee.Image(0).rename('water_mask').updateMask(ee.Image(0)).set('system:time_start', s.millis())
    def _from_s1():
        m = col.median().clip(region_geometry)
        vv = m.select('VV')
        vh = m.select('VH')
        ratio = vv.subtract(vh)  # dB - dB
        # limiares simples e robustos para água (ajuste se necessário):
        # VV <= -17 dB E (VV-VH) <= 5 dB
        water = vv.lte(-17).And(ratio.lte(5)).rename('water_mask')
        return water.set('system:time_start', s.millis())
    return ee.Image(ee.Algorithms.If(col.size().gt(0), _from_s1(), _empty()))

# --- Landsat L2 mensal (2005+) -> água por MNDWI ---
# Coleções L2 (C02) – SR_Bx com escala/offset; QA_PIXEL para nuvens
L5  = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
L7  = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')
L8  = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
L9  = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')

def mask_l2_clouds(img):
    qa = img.select('QA_PIXEL')
    # bits: 1 dilated cloud, 3 cloud, 4 cloud shadow, 5 snow
    mask = (qa.bitwiseAnd(1<<1).eq(0)
            .And(qa.bitwiseAnd(1<<3).eq(0))
            .And(qa.bitwiseAnd(1<<4).eq(0))
            .And(qa.bitwiseAnd(1<<5).eq(0)))
    return img.updateMask(mask)

def scale_l2_reflectance(img):
    return img.multiply(0.0000275).add(-0.2)

def landsat_collection_2005_2025():
    # seleção de bandas padronizada (SR_B3=Green, SR_B6=SWIR1 em todas C02 L2)
    allc = (L5.merge(L7).merge(L8).merge(L9)
            .filterBounds(region_geometry)
            .map(mask_l2_clouds)
            .map(lambda im: im.select(['SR_B3','SR_B6','QA_PIXEL']).copyProperties(im, im.propertyNames()))
            .map(lambda im: im.addBands(scale_l2_reflectance(im.select(['SR_B3','SR_B6'])).rename(['green','swir1']), overwrite=True))
            .map(lambda im: im.clip(region_geometry)))
    return allc

landsat_ic = landsat_collection_2005_2025()

def landsat_water_for_month(iso):
    s = ee.Date(iso); e = s.advance(1, "month")
    col = landsat_ic.filterDate(s, e)
    def _empty():
        return ee.Image(0).rename('water_mask').updateMask(ee.Image(0)).set('system:time_start', s.millis())
    def _from_ls():
        med = col.median()
        green = med.select('green'); swir1 = med.select('swir1')
        mndwi = green.subtract(swir1).divide(green.add(swir1).max(1e-6))
        water = mndwi.gt(0.2).rename('water_mask')   # limiar conservador; ajuste 0.2–0.35 se necessário
        return water.set('system:time_start', s.millis())
    return ee.Image(ee.Algorithms.If(col.size().gt(0), _from_ls(), _empty()))

# --- (Opcional) LANCE NRT flood monthly como Asset próprio (0/1) ---
# Preencha com seu ID se você subiu mosaicos mensais
LANCE_MONTHLY_ASSET = None  # ex.: 'users/SEU_USUARIO/LANCE/monthly'
lance_ic = ee.ImageCollection(LANCE_MONTHLY_ASSET) if LANCE_MONTHLY_ASSET else ee.ImageCollection([])

def lance_water_for_month(iso):
    if not LANCE_MONTHLY_ASSET:
        return ee.Image(0).rename('water_mask').updateMask(ee.Image(0)).set('system:time_start', ee.Date(iso).millis())
    s = ee.Date(iso); e = s.advance(1, "month")
    im = lance_ic.filterDate(s, e).max().rename('water_mask')  # assume 0/1
    return im.set('system:time_start', s.millis())

# Constrói a coleção mensal 2022–2025 a partir de S1 OR Landsat OR LANCE
def water_month_2022_2025(iso):
    s1  = s1_water_for_month(iso)
    ls  = landsat_water_for_month(iso)
    ln  = lance_water_for_month(iso)
    union = s1.max(ls).max(ln).rename('water_mask').set('system:time_start', ee.Date(iso).millis())
    return union

months_2022_2025 = ee.List(make_date_range("2022-01-01", "2025-12-01", "MS"))
water_masks_2022_2025 = ee.ImageCollection(months_2022_2025.map(water_month_2022_2025))

# Coleção completa 2005–2025 (JRC até 2021 + Extensão 2022–2025)
water_monthly_2005_2025 = jrc_water_masks_2005_2021.merge(water_masks_2022_2025).sort("system:time_start")


# -----------------------------
# C) Global Flood Database (2005–2018) + (opcional) DFO 2019–2025
# -----------------------------
gfd_ic = (ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1")
          .filterBounds(region_geometry))
gfd_2005_2018 = gfd_ic.filterDate("2005-01-01", "2018-12-31")

gfd_floodplain = (gfd_2005_2018.select("flooded").sum().gt(0)
                  .rename("gfd_floodplain").clip(region_geometry))

# (Opcional) DFO como Asset (polígonos por evento) para 2019–2025
DFO_ASSET = None  # ex.: 'users/SEU_USUARIO/DFO_floods_1985_2025'
dfo_fc = ee.FeatureCollection(DFO_ASSET) if DFO_ASSET else ee.FeatureCollection([])

# Exemplo: converter DFO por-mês (2019–2025) em raster 0/1 p/ validação mensal
def dfo_month_mask(iso):
    if not DFO_ASSET:
        return ee.Image(0).rename('dfo_mask').updateMask(ee.Image(0)).set('system:time_start', ee.Date(iso).millis())
    s = ee.Date(iso); e = s.advance(1, "month")
    month_evts = dfo_fc.filterDate(s, e).filterBounds(region_geometry)
    # rasterizar (resolução ~ 250 m para ser leve)
    raster = (ee.Image(0).rename('dfo_mask')
              .paint(month_evts, 1)
              .toUint8()
              .set('system:time_start', s.millis()))
    return raster

months_2019_2025 = ee.List(make_date_range("2019-01-01","2025-12-01","MS"))
dfo_monthly_2019_2025 = ee.ImageCollection(months_2019_2025.map(dfo_month_mask))


# -----------------------------
# D) Resumos anuais 2005–2025 (contagem de meses com água)
# -----------------------------
def water_year_sum(year):
    y = ee.Number(year)
    s = ee.Date.fromYMD(y, 1, 1)
    e = s.advance(1, 'year')
    cnt = water_monthly_2005_2025.filterDate(s, e).sum().rename('water_months')
    return cnt.set('year', y).set('system:time_start', s.millis())

years_2005_2025 = ee.List.sequence(2005, 2025)
water_annual_2005_2025 = ee.ImageCollection(years_2005_2025.map(water_year_sum))


# -----------------------------
# E) Validação simples (GFD x água mensal) — janela [m-1, m+2]
# -----------------------------
def validate_one_event(ev_img):
    start = ee.Date(ev_img.get("system:time_start"))
    start_win = start.advance(-1, "month")
    end_win = start.advance(2, "month")

    wm = water_monthly_2005_2025.filterDate(start_win, end_win).max().rename("water_during_event")

    flooded = ev_img.select("flooded").selfMask()
    permanent = ev_img.select("jrc_perm_water").eq(1)  # da própria imagem do GFD
    flooded_only = flooded.updateMask(permanent.Not())

    tp = wm.And(flooded_only)

    stats = tp.addBands(flooded_only).reduceRegion(
        reducer=ee.Reducer.mean().combine(reducer2=ee.Reducer.sum(), sharedInputs=False),
        geometry=region_geometry, scale=30, maxPixels=1e13
    )
    return ee.Feature(None, {
        "event_id": ev_img.get("id"),
        "event_country": ev_img.get("dfo_country"),
        "water_detect_mean": stats.get("water_during_event_mean"),
        "gfd_flood_pixels": stats.get("flooded_sum")
    })

validation_fc = ee.FeatureCollection(gfd_2005_2018.limit(10).map(validate_one_event))


In [20]:
# Harmoniza Land Cover para sempre usar 'lc_majority'
def build_lc_series_2005_2025():
    def one_year(y):
        y = ee.Number(y)
        s = ee.Date.fromYMD(y, 1, 1)
        e = s.advance(1, 'year')

        # Pega a primeira imagem do ano (MCD12Q1 v061)
        im = ee.ImageCollection('MODIS/061/MCD12Q1').filterDate(s, e).first()
        im = ee.Image(ee.Algorithms.If(im, im, ee.Image.constant(255).rename('lc_majority')))

        bands = im.bandNames()

        # Seleciona a banda correta e renomeia para 'lc_majority'
        lc = ee.Image(ee.Algorithms.If(
            bands.contains('Majority_Land_Cover_Type_1'),
            im.select('Majority_Land_Cover_Type_1').rename('lc_majority'),
            ee.Image(ee.Algorithms.If(
                bands.contains('LC_Type1'),  # fallback p/ v006 se aparecer em algum cache
                im.select('LC_Type1').rename('lc_majority'),
                ee.Image(ee.Algorithms.If(
                    bands.contains('Classification'),  # fallback p/ coleções derivadas
                    im.select('Classification').rename('lc_majority'),
                    ee.Image.constant(255).rename('lc_majority')  # fallback final
                ))
            ))
        ))

        return lc.set('year', y)

    years = ee.List.sequence(2005, 2024)
    ic = ee.ImageCollection(years.map(one_year))

    # Projeção de 2025 = 2024 (para cobrir o intervalo completo)
    lc2024 = ic.filter(ee.Filter.eq('year', 2024)).first()
    lc2025 = ee.Image(lc2024).set('year', 2025)
    return ic.merge(ee.ImageCollection([lc2025]))

# Recria e sobrescreve a coleção global usada na Seção 8
land_cover_series_ic = build_lc_series_2005_2025()

# Sanidade: deve retornar apenas ['lc_majority']
print("Bandas LC (amostra 2015):",
      ee.Image(land_cover_series_ic.filter(ee.Filter.eq('year', 2015)).first()).bandNames().getInfo())


Bandas LC (amostra 2015): ['lc_majority']


In [21]:
# ============================================================
# SEÇÃO 8 — Feature Engineering & Hazard Logic (2005–2025)
# ============================================================

import os, json, glob, re, csv
import ee, geemap

# ----------------------------------------------
# 0) Pastas (use os GeoJSONs já simplificados)
# ----------------------------------------------
GEO_DIR = "./flui_app/data_manager/geojson_simplified"
OUT_DIR = "./features/municipal_monthly"
os.makedirs(OUT_DIR, exist_ok=True)

# ----------------------------------------------
# 1) Earth Engine
# ----------------------------------------------
try:
    ee.Image(1).getInfo()
    print("EE OK.")
except Exception:
    ee.Initialize()
    print("EE inicializado.")

# ----------------------------------------------
# 2) Utilitários
# ----------------------------------------------
def make_date_range(start_date, end_date, freq):
    import pandas as pd
    dates = pd.date_range(start=start_date, end=end_date, freq=freq, inclusive="both")
    return [str(d.date()) for d in dates]

MONTHS = make_date_range("2005-01-01", "2025-12-01", "MS")

def _first_or_fill(ic, start, end, band_names, fill_value=0):
    col = ic.filterDate(start, end)
    cond = col.size().gt(0)
    def _filled():
        fills = [ee.Image.constant(fill_value).rename(b) for b in band_names]
        return ee.Image.cat(fills)
    img_first = ee.Image(col.first())
    return ee.Image(ee.Algorithms.If(cond, img_first.select(band_names), _filled()))

def _strip_props(feat, keep=('id','name','nome','NM_MUN','CD_MUN','cd_mun','SIGLA_UF','UF','uf')):
    props = feat.get('properties', {}) or {}
    kept = {k: props[k] for k in props if k in keep}
    if 'uf' in kept and 'UF' not in kept:
        kept['UF'] = str(kept['uf']).upper()
    out = {'type':'Feature','properties': kept, 'geometry': feat.get('geometry', None)}
    fid = feat.get('id', None)
    if fid is None:
        for k in ('CD_MUN','cd_mun','id'):
            if k in props:
                fid = str(props[k]); break
    if fid is not None:
        out['id'] = str(fid)
    return out

def load_state_municipal_fc(state_geojson_path):
    with open(state_geojson_path, "r", encoding="utf-8") as f:
        gj = json.load(f)
    feats = gj.get('features', [])
    new_feats = [_strip_props(ft) for ft in feats]
    gj_min = {'type': 'FeatureCollection', 'features': new_feats}
    return geemap.geojson_to_ee(gj_min)

# ----------------------------------------------
# 3) Land cover para um ano (sem LC_Type1)
#    - Prioriza MCD12Q1 v061: Majority_Land_Cover_Type_1
#    - Fallback v006: LC_Type1
#    - Sempre renomeia p/ 'lc_majority'
# ----------------------------------------------
def lc_for_year(year):
    year = ee.Number(year)
    s = ee.Date.fromYMD(year, 1, 1)
    e = s.advance(1, 'year')

    lc061 = ee.ImageCollection("MODIS/061/MCD12Q1").filterDate(s, e).first()
    lc006 = ee.ImageCollection("MODIS/006/MCD12Q1").filterDate(s, e).first()

    def _from_061():
        img = ee.Image(lc061)
        bands = img.bandNames()
        return ee.Image(ee.Algorithms.If(
            bands.contains('Majority_Land_Cover_Type_1'),
            img.select('Majority_Land_Cover_Type_1').rename('lc_majority'),
            ee.Image.constant(255).rename('lc_majority')
        ))

    def _from_006():
        img = ee.Image(lc006)
        bands = img.bandNames()
        return ee.Image(ee.Algorithms.If(
            bands.contains('LC_Type1'),
            img.select('LC_Type1').rename('lc_majority'),
            ee.Image.constant(255).rename('lc_majority')
        ))

    return ee.Image(ee.Algorithms.If(lc061, _from_061(),
                  ee.Algorithms.If(lc006, _from_006(),
                                   ee.Image.constant(255).rename('lc_majority'))))

# ----------------------------------------------
# 4) Pilha mensal de variáveis (usa lc_for_year)
# ----------------------------------------------
def monthly_feature_stack(start_iso):
    s = ee.Date(start_iso)
    e = s.advance(1, "month")
    year = s.get('year')

    # Dinâmicos mensais
    gpm_band  = ee.Image(gpm_monthly.first()).bandNames().get(0)  # ex.: 'precipitation'
    rain_month = _first_or_fill(gpm_monthly, s, e, [gpm_band]).rename('rain_sum')
    sm_month   = _first_or_fill(soil_moisture_monthly, s, e, ['soil_moisture'])
    tws_month  = _first_or_fill(grace_tws_monthly,     s, e, ['tws_cm'])
    water_m    = _first_or_fill(water_monthly_2005_2025, s, e, ['water_mask'])
    radar_m    = _first_or_fill(radar_monthly,        s, e, ['sar_ratio_db','sar_entropy','sar_energy'])

    # Semidínamicos anuais
    pop_year = population_series_ic.filter(ee.Filter.eq('year', year)).first()
    pop_year = ee.Image(ee.Algorithms.If(pop_year, pop_year.select('pop'),
                                         ee.Image.constant(0).rename('pop')))

    lc_year  = lc_for_year(year)  # <- sempre 'lc_majority'

    imperv_year = impervious_series_ic.filter(ee.Filter.eq('year', year)).first()
    imperv_year = ee.Image(ee.Algorithms.If(
        imperv_year,
        ee.Image(imperv_year).select('impervious_fraction'),
        ee.ImageCollection(impervious_series_ic).mean().select('impervious_fraction')
    )).rename('impervious_fraction')

    # Estáticos
    slope_lt5 = slope_deg.lt(5).rename('slope_lt5')
    stack_static = ee.Image.cat([
        hand.rename('hand_m'),
        twi.rename('twi'),
        distance_to_channel.rename('dist_to_channel_m'),
        water_recurrence.rename('water_recurrence_pct')
    ])

    return ee.Image.cat([
        rain_month, sm_month, tws_month, water_m, radar_m,
        pop_year.rename('pop'),
        lc_year,                    # 'lc_majority'
        imperv_year,
        slope_lt5, stack_static
    ]).set('system:time_start', s.millis()).set('yyyymm', s.format('YYYYMM'))

# ----------------------------------------------
# 5) Export local em CHUNKS (sem geometria)
# ----------------------------------------------
def _fc_to_csv_chunks(fc, out_csv, chunk_size=150):
    fc_nogeo = ee.FeatureCollection(fc.map(lambda f: ee.Feature(f).setGeometry(None)))
    n = fc_nogeo.size().getInfo()
    if n == 0:
        print(f"↪ Coleção vazia, pulando {out_csv}")
        return

    cols = [
        'yyyymm','uf','id','CD_MUN','cd_mun','NM_MUN','nome','SIGLA_UF','UF',
        'pop_mean','rain_sum_mean','soil_moisture_mean','tws_cm_mean','water_mask_mean',
        'impervious_fraction_mean','slope_lt5_mean',
        'hand_m_mean','twi_mean','dist_to_channel_m_mean','water_recurrence_pct_mean',
        'lc_majority_mode',
        'sar_ratio_db_mean','sar_entropy_mean','sar_energy_mean'
    ]
    write_header = not os.path.exists(out_csv)
    with open(out_csv, "a", newline="", encoding="utf-8") as fcsv:
        writer = csv.DictWriter(fcsv, fieldnames=cols, extrasaction='ignore')
        if write_header:
            writer.writeheader()
        for offset in range(0, n, chunk_size):
            sub = ee.FeatureCollection(fc_nogeo.toList(chunk_size, offset))
            sub_info = sub.getInfo()
            feats = sub_info.get('features', [])
            for ft in feats:
                writer.writerow(ft.get("properties", {}) or {})
            print(f"   • chunk {offset}-{min(offset+chunk_size, n)} / {n} → {out_csv}")

def _append_fc_no_geom(fc, out_csv, header_cols):
    """
    Converte um FeatureCollection pequeno (chunk) sem geometria e apenda no CSV.
    Usa getInfo somente no sublote (pequeno), evitando estouro de memória.
    """
    fc_nogeo = ee.FeatureCollection(fc.map(lambda f: ee.Feature(f).setGeometry(None)))
    info = fc_nogeo.getInfo()
    feats = info.get('features', [])

    write_header = not os.path.exists(out_csv)
    with open(out_csv, "a", newline="", encoding="utf-8") as fcsv:
        writer = csv.DictWriter(fcsv, fieldnames=header_cols, extrasaction='ignore')
        if write_header:
            writer.writeheader()
        for ft in feats:
            writer.writerow(ft.get("properties", {}) or {})


# ----------------------------------------------
# 6) Redução municipal por UF × mês → CSV
# ----------------------------------------------
def reduce_state_month_to_csv_chunked(state_file,
                                      start_iso,
                                      scale=6000,           # ↑ deixe a redução mais leve (5–8 km)
                                      tile_scale=16,        # ↑ mais tiles menores = menos memória por tile
                                      mun_chunk_size=60):   # nº de municípios por sublote
    import re, os
    m = re.search(r'geojs\-BR([A-Z]{2})\-mun\.json$', state_file)
    uf = m.group(1) if m else 'UF'
    s = ee.Date(start_iso)
    yyyymm = s.format('YYYYMM').getInfo()

    print(f"[{uf}] {yyyymm} → carregando municípios ({state_file}) ...")
    munic_fc = load_state_municipal_fc(state_file).filterBounds(region_geometry)
    n_mun = munic_fc.size().getInfo()
    if n_mun == 0:
        print(f"[{uf}] {yyyymm} → 0 municípios no AOI, pulando.")
        return

    # imagem do mês
    img = monthly_feature_stack(start_iso)

    # bandas p/ média (booleans viram fração)
    mean_bands = [
        'rain_sum','soil_moisture','tws_cm','water_mask',
        'sar_ratio_db','sar_entropy','sar_energy',
        'pop','impervious_fraction','slope_lt5',
        'hand_m','twi','dist_to_channel_m','water_recurrence_pct'
    ]
    img_mean = img.select(mean_bands).unmask(0)
    lc_img   = img.select(['lc_majority']).unmask(255)

    # cabeçalho fixo do CSV
    out_csv = os.path.join(OUT_DIR, f"features_{uf}_{yyyymm}.csv")
    header_cols = [
        'yyyymm','uf','id','CD_MUN','cd_mun','NM_MUN','nome','SIGLA_UF','UF',
        'pop_mean','rain_sum_mean','soil_moisture_mean','tws_cm_mean','water_mask_mean',
        'impervious_fraction_mean','slope_lt5_mean',
        'hand_m_mean','twi_mean','dist_to_channel_m_mean','water_recurrence_pct_mean',
        'lc_majority_mode',
        'sar_ratio_db_mean','sar_entropy_mean','sar_energy_mean'
    ]

    print(f"[{uf}] {yyyymm} → {n_mun} municípios; processando em chunks de {mun_chunk_size} ...")
    # pagina os municípios
    for offset in range(0, n_mun, mun_chunk_size):
        sub = ee.FeatureCollection(munic_fc.toList(mun_chunk_size, offset))

        # reduceRegions no sublote
        fc_mean = img_mean.reduceRegions(
            collection=sub,
            reducer=ee.Reducer.mean(),
            scale=scale,
            tileScale=tile_scale
        )
        fc_lc = lc_img.reduceRegions(
            collection=sub,
            reducer=ee.Reducer.mode(),
            scale=scale,
            tileScale=tile_scale
        )

        # join por system:index
        joined = ee.Join.inner().apply(
            primary=fc_mean,
            secondary=fc_lc,
            condition=ee.Filter.equals(leftField='system:index', rightField='system:index')
        )
        def _merge_feats(j):
            left  = ee.Feature(j.get('primary'))
            right = ee.Feature(j.get('secondary'))
            return left.copyProperties(right, right.propertyNames())

        merged_sub = ee.FeatureCollection(joined).map(_merge_feats) \
                        .map(lambda f: f.set({'yyyymm': yyyymm, 'uf': uf}))

        # apenda este sublote no CSV (sem geometria)
        _append_fc_no_geom(merged_sub, out_csv, header_cols)
        print(f"   • [{uf}] {yyyymm} chunk {offset}-{min(offset+mun_chunk_size, n_mun)} salvo.")

    print(f"[{uf}] {yyyymm} ✔ CSV consolidado: {out_csv}")

# ----------------------------------------------
# 7) Quicklook — mapa reduzido (viz leve)
# ----------------------------------------------
def quicklook_map(start_iso, viz_scale=3000):
    s = ee.Date(start_iso)
    img = monthly_feature_stack(start_iso)
    proj_viz = ee.Projection('EPSG:3857').atScale(viz_scale)

    rain_v = img.select('rain_sum').reproject(proj_viz)
    imp_v  = img.select('impervious_fraction').reproject(proj_viz)
    lc_v   = img.select('lc_majority').reproject(proj_viz)

    rain_vis = {'min': 0, 'max': 500, 'palette': ['000044','3366ff','66ccff','ccffff']}
    imp_vis  = {'min': 0, 'max': 1.0, 'palette': ['ffffff','ffcc00','ff6600','cc0000']}
    lc_vis   = {'min': 0, 'max': 16, 'palette': [
        '152106','225129','369b47','30eb5b','387242','6a2325',
        'c3aa69','b76031','d9903d','91af40','111149','cdb33b',
        'cc0013','33280d','d7cdcc','f7e084','6f6f6f'
    ]}

    M = geemap.Map()
    M.center_object(region_geometry, 4)
    M.addLayer(rain_v, rain_vis, f"Rain (mm) {s.format('YYYY-MM').getInfo()}")
    M.addLayer(imp_v,  imp_vis,  f"Impervious {s.format('YYYY').getInfo()}")
    M.addLayer(lc_v,   lc_vis,   f"LC (IGBP) {s.format('YYYY').getInfo()}")
    return M


EE OK.


In [23]:
# Teste: RS, jan/2015 com escala bem leve e chunks pequenos
state_files = sorted(glob.glob(os.path.join(GEO_DIR, "geojs-BR??-mun.json")))
test_state = None
for p in state_files:
    if p.endswith("geojs-BRRS-mun.json"):
        test_state = p; break
if test_state is None and len(state_files) > 0:
    test_state = state_files[0]

reduce_state_month_to_csv_chunked(
    test_state,
    "2015-01-01",
    scale=30000,        # ↑ comece grosso; se passar, tente 5000 ou 4000
    tile_scale=32,     # ↑ mais seguro contra memória
    mun_chunk_size=1  # ↓ chunks menores = mais chamadas, porém mais estável
)


[RS] 201501 → carregando municípios (./flui_app/data_manager/geojson_simplified\geojs-BRRS-mun.json) ...
[RS] 201501 → 496 municípios; processando em chunks de 1 ...


EEException: User memory limit exceeded.